In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import re

Testing lem:equilibrium-expressed-opinions-case2 on $n=5$ matrix.

## Fixed Parameters

In [ ]:
# Desides dimensions of blocks
n=5
alpha = 3/5
k=1

# Does not determine dimension
beta = 0.2
gamma = 0.1
delta = 0.25
mathbf_e_top_s = 1

# With numpy

## Pre analysis
Already now, we know that $\bar{\mathbf{s}}_M = \frac{\alpha+\delta}{\alpha n}\mathbf{e}^\top\mathbf{s} = \frac{\frac{3}{5}+0.25}{3}1 = 0.28...$

In [ ]:
bar_s_M = (alpha+delta)/(alpha*n)*mathbf_e_top_s
print("bar_s_M:")
print(bar_s_M)

bar_s_M_prim = (1-alpha-delta)/((1-alpha)*n)*mathbf_e_top_s
print("bar_s_M:")
print(bar_s_M)

and 

$(1+\gamma)\bar{\mathbf{s}}_M = 1.1\cdot 0.28... = 0.31...$

ok to not truncate!

In [ ]:
z_M = (1+gamma)*bar_s_M
print(z_M)
assert z_M <= 1

In [ ]:
n_1 = int(alpha * n)

n_2 = int((1-alpha) * n)

print(n_1, n_2, k)

From description of graph:

$\mathbf{W} = \begin{pmatrix}
    0 & 1 & 1 & 1 & 0\\
    1 & 0 & 1 & 0 & 0\\
    1 & 1 & 0 & 0 & 0\\
    1 & 0 & 0 & 0 & 1\\
    0 & 0 & 0 & 1 & 0
\end{pmatrix}$

In [ ]:
mathbf_W = np.array([
    [0, 1, 1, 1, 0],
    [1, 0, 1, 0, 0],
    [1, 1, 0, 0, 0],
    [1, 0, 0, 0, 1],
    [0, 0, 0, 1, 0]
])

print(mathbf_W)

$\mathbf{D} = \begin{pmatrix}
    3 & 0 & 0 & 0 & 0\\
    0 & 2 & 0 & 0 & 0\\
    0 & 0 & 2 & 0 & 0\\
    0 & 0 & 0 & 2 & 0\\
    0 & 0 & 0 & 0 & 1
\end{pmatrix}$

In [ ]:
mathbf_D = np.array([
    [3, 0, 0, 0, 0],
    [0, 2, 0, 0, 0],
    [0, 0, 2, 0, 0],
    [0, 0, 0, 2, 0],
    [0, 0, 0, 0, 1]
])
mathbf_D

In [ ]:
mathbf_I = np.eye(n)
mathbf_I




$\mathbf{I} = \begin{pmatrix}
    1 & 0 & 0 & 0 & 0\\
    0 & 1 & 0 & 0 & 0\\
    0 & 0 & 1 & 0 & 0\\
    0 & 0 & 0 & 1 & 0\\
    0 & 0 & 0 & 0 & 1
\end{pmatrix}$



$\mathbf{L} = \mathbf{D}-\mathbf{W}$




In [ ]:
mathbf_L = mathbf_D - mathbf_W
mathbf_L

From Theorem 1:

$\mathbf{M} = (1+\beta)\mathbf{I} + \beta \mathbf{D} + \mathbf{L}$

In [ ]:
mathbf_M = (1+beta) * mathbf_I + beta * mathbf_D + mathbf_L
mathbf_M

## Invert M^-1 with numpy straight from M

Correct M_inv:

In [ ]:
mathbf_M_inv = np.linalg.inv(mathbf_M)
mathbf_M_inv

## vector-matrix-vector multiplication with numpy

In [ ]:
mathbf_e_top_M = np.block([
    [np.ones((1,n_1)), np.zeros((1,n_2))]
])
mathbf_e_top_M_prim = np.block([
    [np.zeros((1,n_1)), np.ones((1,n_2))]
])
print("mathbf_e_top_M:")
print(mathbf_e_top_M)

print("mathbf_e_top_M_prim:")
print(mathbf_e_top_M_prim)

In [ ]:
e_top_M_inv = mathbf_e_top_M @ mathbf_M_inv
print("e_top_M_inv:")
print(e_top_M_inv)

#Taking the scalars only from the first index in each block.
U_1 = e_top_M_inv[0, 0]
print("U_1:")
print(U_1)
U_2 = e_top_M_inv[0, k]
V_1 = e_top_M_inv[0, n_1]
V_2 = e_top_M_inv[0, n_1+k]

e_top_M_prim_inv = mathbf_e_top_M_prim @ mathbf_M_inv
print("e_top_M_prim_inv:")
print(e_top_M_prim_inv)

X_1 = e_top_M_prim_inv[0, 0]
print("X_1:")
print(X_1)
X_2 = e_top_M_prim_inv[0, k]
Y_1 = e_top_M_prim_inv[0, n_1]
Y_2 = e_top_M_prim_inv[0, n_1+k]

# Opinion vectors

In [ ]:
zeta = np.block([
    (1+gamma)*(alpha+delta)/(n_1) * np.ones((k)),
    (1+gamma)*(alpha+delta)/(n_1) * np.ones((n_1-k)),
    (1-gamma)*(1-alpha-delta)/(n_2) * np.ones((k)),
    (1-gamma)*(1-alpha-delta)/(n_2) * np.ones((n_2-k))
]) * mathbf_e_top_s
print(zeta)

In [ ]:
# "Made up" s vector with avarage in each entry

mathbf_s = np.block([
    bar_s_M * np.ones((n_1)),
    bar_s_M_prim * np.ones((n_2))
])
print("mathbf_s:")
print(mathbf_s)

In [ ]:
first_term  = e_top_M_inv @ (mathbf_s)
second_term = e_top_M_inv @ (beta*(mathbf_I + mathbf_D) @ zeta)


e_M_top_tilde_z_equi      = e_top_M_inv      @ (mathbf_s + beta*(mathbf_I + mathbf_D) @ zeta)

e_M_prim_top_tilde_z_equi = e_top_M_prim_inv @ (mathbf_s + beta*(mathbf_I + mathbf_D) @ zeta)

print("e_M_top_tilde_z_equi:")
print(e_M_top_tilde_z_equi)

sum = e_M_top_tilde_z_equi + e_M_prim_top_tilde_z_equi
print("sum:")
print(sum)

# With theorem

In [ ]:
#thm:sum-of-expressed-equilibrium-case2-nontrunc
def thm_sum_of_expressed_equilibrium_case2(alpha, beta, gamma, delta, k, n, e_top_s, trunc = False, verify=False):

    n_1 = int(alpha * n)
    n_2 = int((1 - alpha) * n)

    a_1 = 2.0 + beta + (beta + 1) * n_1
    d_1 = 2.0 + beta + (beta + 1) * n_2
    a_2 = 1.0+(1+beta) * n_1
    d_2 = 1.0+(1+beta) * n_2

    A = a_1*a_2-a_2*k-a_1*(n_1-k)
    D = d_1*d_2-d_2*k-d_1*(n_2-k)

    p_a_nom = (1+a_2/(a_1*A))*a_1**2
    p_d_nom = (1+d_2/(d_1*D))*d_1**2
    p_a_denom = (a_1*d_1-1)*(a_1*d_1-1-a_1*(1+a_2/(a_1*A))*k)
    p_d_denom = (d_1*a_1-1)*(d_1*a_1-1-d_1*(1+d_2/(d_1*D))*k)
    p_a = p_a_nom / p_a_denom
    p_d = p_d_nom / p_d_denom
    
    s_a = 1/(d_2*(d_2-(n_2-k)))
    s_d = 1/(a_2*(a_2-(n_1-k)))

    P_a_nom = (a_1 * k +(a_1*d_1-1)*(p_a*k**2+1))*(a_1*d_1-1)
    P_d_nom = (d_1 * k +(d_1*a_1-1)*(p_d*k**2+1))*(d_1*a_1-1)
    P_a_denom = (a_1*d_1-1)*((a_1*d_1-1)*d_2**2-d_2*(a_1*k +(a_1*d_1-1)*(p_a*k**2+1))*(n_2-k))
    P_d_denom = (d_1*a_1-1)*((d_1*a_1-1)*a_2**2-a_2*(d_1*k +(d_1*a_1-1)*(p_d*k**2+1))*(n_1-k))
    P_a = P_a_nom / P_a_denom
    P_d = P_d_nom / P_d_denom

    S_a_nom = a_1**2*((1+a_2/(a_1*A))*d_2+(n_2-k)+(n_2-k)**2*s_a*d_2)*d_2
    S_d_nom = d_1**2*((1+d_2/(d_1*D))*a_2+(n_1-k)+(n_1-k)**2*s_d*a_2)*a_2
    S_a_denom = (a_1*d_1-1)*d_2*(d_2*(a_1*d_1-1) -a_1*k*((1+a_2/(a_1*A))*d_2+(n_2-k)+(n_2-k)**2*s_a*d_2))
    S_d_denom = (d_1*a_1-1)*a_2*(a_2*(d_1*a_1-1) -d_1*k*((1+d_2/(d_1*D))*a_2+(n_1-k)+(n_1-k)**2*s_d*a_2))
    S_a = S_a_nom / S_a_denom
    S_d = S_d_nom / S_d_denom

    Q_a=a_1/((a_1*d_1-1)*d_2)+(a_1*P_a*(n_2-k))/(a_1*d_1-1)+(p_a*k)/(d_2)+k*(n_2-k)*p_a*P_a
    Q_d=d_1/((d_1*a_1-1)*a_2)+(d_1*P_d*(n_1-k))/(d_1*a_1-1)+(p_d*k)/(a_2)+k*(n_1-k)*p_d*P_d

    v = a_1/(a_1*d_1-1)+S_a*k 
    
    #For verification. Same as in lemma statement.
    if verify:
        thm_mathbf_U = np.block([
            [(d_1/(d_1*a_1-1))*np.eye(k) + S_d*np.ones((k,k)), Q_d * np.ones((k, n_1-k))],
            [Q_d*np.ones((n_1-k, k)), 1/a_2 * np.eye(n_1-k) + P_d*np.ones((n_1-k, n_1-k))]
        ])
        thm_mathbf_V = 1/(a_1*a_2*A) * np.block([
            [(a_2*A+a_2**2)*v*np.ones((k,k)), (a_2*A+a_2**2)*Q_a*k*np.ones((k, n_2-k))],
            [a_1*a_2*v*np.ones((n_1-k,k)), a_1*a_2*Q_a*k*np.ones((n_1-k,n_2-k))]
        ])
        thm_mathbf_X = np.transpose(thm_mathbf_V)
        thm_mathbf_Y = np.block([
            [a_1/(a_1*d_1-1)*np.eye(k)+S_a*np.ones((k,k)), Q_a*np.ones((k,n_2-k))],
            [Q_a*np.ones((n_2-k,k)), 1/d_2*np.eye(n_2-k)+P_a*np.ones((n_2-k,n_2-k))]
        ])
    
        thm_mathbf_M_inv = np.block([
            [thm_mathbf_U, thm_mathbf_V],
            [thm_mathbf_X, thm_mathbf_Y]
        ])
        print("thm_mathbf_M_inv:")
        print(thm_mathbf_M_inv)
        assert np.allclose(thm_mathbf_M_inv, mathbf_M_inv), "thm_mathbf_M_inv incorrect"
    
    thm_U_1 = d_1/(d_1*a_1-1) + k*S_d+(n_1-k)*Q_d
    thm_U_2 = 1/a_2 +(n_1-k)*P_d +k*Q_d
    thm_V_1 = (k*v*(a_2*A+a_2**2) + (n_1-k)*a_1*a_2*v)/(a_1*a_2*A)
    thm_V_2 = (k**2*Q_a*(a_2*A+a_2**2)+a_1*a_2*Q_a*k*(n_1-k))/(a_1*a_2*A)

    if verify:
        print("thm_U_1:")
        print(thm_U_1)
        assert np.isclose(thm_U_1, U_1), "thm_U_1 incorrect"
        print("thm_U_2:")
        print(thm_U_2)
        assert np.isclose(thm_U_2, U_2), "thm_U_2 incorrect"
        print("thm_V_1:")
        print(thm_V_1)
        assert np.isclose(thm_V_1, V_1), "thm_V_1 incorrect"
        print("thm_V_2:")
        print(thm_V_2)
        assert np.isclose(thm_U_2, U_2), "thm_U_2 incorrect"
    
    thm_X_1 = (k*v*(a_2*A+a_2**2) + k*(n_2-k)*Q_a*(a_2*A+a_2**2))/(a_1*a_2*A)
    thm_X_2 = (k*a_1*a_2*v +(n_2-k)*a_1*a_2*Q_a*k)/(a_1*a_2*A)
    thm_Y_1 = a_1/(a_1*d_1-1) + k*S_a + (n_2-k)*Q_a
    thm_Y_2 = k*Q_a + 1/d_2+(n_2-k)*P_a
    
    if verify:
        thm_e_top_M_inv = np.block([
            [thm_U_1 * np.ones((1,k)), thm_U_2 * np.ones((1,n_1-k)), thm_V_1 * np.ones((1,k)), thm_V_2 * np.ones((1,n_2-k))]
        ])
        print("thm_e_top_M_inv:")
        print(thm_e_top_M_inv)
        assert np.allclose(thm_e_top_M_inv, e_top_M_inv), "thm_e_top_M_inv incorrect"
    
    
        print("X_1:")
        print(X_1)
        print("thm_X_1:")
        print(thm_X_1)
        assert np.isclose(thm_X_1, X_1), "thm_X_1 incorrect"
        print("X_2:")
        print(X_2)
        print("thm_X_2:")
        print(thm_X_2)
        assert np.isclose(thm_X_2, X_2), "thm_X_2 incorrect"
        print("Y_1:")
        print(Y_1)
        print("thm_Y_1:")
        print(thm_Y_1)
        assert np.isclose(thm_Y_1, Y_1), "thm_Y_1 incorrect"
        print("Y_2:")
        print(Y_2)
        print("thm_Y_2:")
        print(thm_Y_2)
        assert np.isclose(thm_Y_2, Y_2), "thm_Y_2 incorrect"
    
        #For verification
        thm_e_top_M_prim_inv = np.block([
            [thm_X_1 * np.ones((1,k)), thm_X_2 * np.ones((1,n_1-k)), thm_Y_1 * np.ones((1,k)), thm_Y_2 * np.ones((1,n_2-k))]
        ])
        print("thm_e_top_M_prim_inv:")
        print(thm_e_top_M_prim_inv)
        assert np.allclose(thm_e_top_M_prim_inv, e_top_M_prim_inv), "thm_e_top_M_prim_inv incorrect"
    
    
    # Opinion vectors
    thm_z_M =      (1+gamma)*(  alpha+delta)/n_1 * e_top_s
    thm_z_M_prim = (1-gamma)*(1-alpha-delta)/n_2 * e_top_s
    
    thm_bar_s_M =      (alpha+delta)/n_1 * e_top_s
    thm_bar_s_M_prim = (1-alpha-delta)/(n_2) * e_top_s
    
    
    if verify:
        #First term
        thm_first_term = U_1*k      *(thm_bar_s_M     ) + \
                         U_2*(n_1-k)*(thm_bar_s_M     ) + \
                         V_1*k      *(thm_bar_s_M_prim) + \
                         V_2*(n_2-k)*(thm_bar_s_M_prim)
        
        print("thm_first_term:")
        print(thm_first_term)
        assert np.isclose(thm_first_term, first_term), "thm_first_term incorrect"
        
        #Second term
        thm_second_term =  U_1*k      * (beta*thm_z_M     *(1+n_1)) + \
                           U_2*(n_1-k)* (beta*thm_z_M     *n_1    ) + \
                           V_1*k      * (beta*thm_z_M_prim*(1+n_2)) + \
                           V_2*(n_2-k)* (beta*thm_z_M_prim*n_2    )
        
        print("thm_second_term:")
        print(thm_second_term)
        assert np.isclose(thm_second_term, second_term), "thm_second_term incorrect"

    #Total
    thm_e_M_top_tilde_z_equi      = U_1*k      *(thm_bar_s_M      + beta*thm_z_M     *(1+n_1)) + \
                                    U_2*(n_1-k)*(thm_bar_s_M      + beta*thm_z_M     *n_1    ) + \
                                    V_1*k      *(thm_bar_s_M_prim + beta*thm_z_M_prim*(1+n_2)) + \
                                    V_2*(n_2-k)*(thm_bar_s_M_prim + beta*thm_z_M_prim*n_2    )
    
    thm_e_M_prim_top_tilde_z_equi = X_1*k      *(thm_bar_s_M      + beta*thm_z_M     *(1+n_1)) + \
                                    X_2*(n_1-k)*(thm_bar_s_M      + beta*thm_z_M     *n_1    ) + \
                                    Y_1*k      *(thm_bar_s_M_prim + beta*thm_z_M_prim*(1+n_2)) + \
                                    Y_2*(n_2-k)*(thm_bar_s_M_prim + beta*thm_z_M_prim*n_2    )
    
    if verify:
        print("e_M_top_tilde_z_equi:")
        print(e_M_top_tilde_z_equi)
        print("thm_e_M_top_tilde_z_equi:")
        print(thm_e_M_top_tilde_z_equi)
        assert np.isclose(thm_e_M_top_tilde_z_equi, e_M_top_tilde_z_equi), "thm_e_M_top_tilde_z_equi incorrect"
        
        print("e_M_prim_top_tilde_z_equi:")
        print(e_M_prim_top_tilde_z_equi)
        print("thm_e_M_prim_top_tilde_z_equi:")
        print(thm_e_M_prim_top_tilde_z_equi)
        assert np.isclose(thm_e_M_prim_top_tilde_z_equi, e_M_prim_top_tilde_z_equi), "thm_e_M_prim_top_tilde_z_equi incorrect"
        
        print("All asserts passed!")

    return (thm_e_M_top_tilde_z_equi, thm_e_M_prim_top_tilde_z_equi)

thm_mathbf_M_inv = thm_sum_of_expressed_equilibrium_case2(alpha, beta, gamma, delta, k, n, mathbf_e_top_s, trunc=False, verify=True)




In [ ]:
def thm_sum_of_expressed_equilibrium_case2_M(alpha, beta, gamma, delta, k, n, e_top_s, trunc = False, verify=False):
    return thm_sum_of_expressed_equilibrium_case2(alpha, beta, gamma, delta, k, n, e_top_s, trunc, verify)[0]
    
def thm_sum_of_expressed_equilibrium_case2_M_prim(alpha, beta, gamma, delta, k, n, e_top_s, trunc = False, verify=False):
    return thm_sum_of_expressed_equilibrium_case2(alpha, beta, gamma, delta, k, n, e_top_s, trunc, verify)[1]

def thm_sum_of_expressed_equilibrium_case2_tot(alpha, beta, gamma, delta, k, n, e_top_s, trunc = False, verify=False):
    return thm_sum_of_expressed_equilibrium_case2(alpha, beta, gamma, delta, k, n, e_top_s, trunc, verify)[0] +\
           thm_sum_of_expressed_equilibrium_case2(alpha, beta, gamma, delta, k, n, e_top_s, trunc, verify)[1]

# Plotting

Copied from `plots.py` and added `trunc`

In [ ]:
def sanitize_label(label):
    label = re.sub(r'mathbf', '', label)
    label = re.sub(r'tilde', '', label)
    return re.sub(r'[^a-zA-Z0-9_-]', '', label)

# Comparing up to four functions agains some x_values
def cmp_funcs(alpha, beta, gamma, delta, k, n, e_top_s, trunc,
                       x_values, x_label, 
                       T_func1, T_func1_label, T_func2=None, T_func2_label=None, 
                       T_func3=None, T_func3_label=None, T_func4=None, T_func4_label=None,
                       yscale='linear'):
    print("x_values:")
    print(x_values)
    
    print("T_func1:")
    print(T_func1)

    y_values1 = T_func1(alpha, beta, gamma, delta, k, n, e_top_s, trunc)
    print("y_values1:")
    print(y_values1)

    plt.plot(x_values, y_values1, label=T_func1_label, linestyle='--')
    if (T_func2):
        y_values2 = T_func2(alpha, beta, gamma, delta, k, n, e_top_s, trunc)
        plt.plot(x_values, y_values2, label=T_func2_label, linestyle='-.')
    if (T_func3):
        y_values3 = T_func3(alpha, beta, gamma, delta, k, n, e_top_s, trunc)
        plt.plot(x_values, y_values3, label=T_func3_label, linestyle=':')
    if (T_func4):
        y_values4 = T_func4(alpha, beta, gamma, delta, k, n, e_top_s, trunc)
        plt.plot(x_values, y_values4, label=T_func4_label, linestyle='-')
    plt.xlabel(x_label)
    plt.ylabel(r'$e^\top z$')
    plt.yscale(yscale)
    plt.grid(True)
    plt.legend()
    plt.tight_layout()

    filename = f"{sanitize_label(T_func1_label)}"
    if T_func2:
        filename += f"_and_{sanitize_label(T_func2_label)}"
    if T_func3:
        filename += f"_and_{sanitize_label(T_func3_label)}"
    if T_func4:
        filename += f"_and_{sanitize_label(T_func4_label)}"
    filename += f"-vs-{sanitize_label(x_label)}"

    if not isinstance(alpha, np.ndarray):
        filename += f"-alpha{alpha}"
    if not isinstance(beta, np.ndarray):
        filename += f"-beta{beta}"
    if not isinstance(gamma, np.ndarray):
        filename += f"-gamma{gamma}"
    if trunc:
        filename += f"-trunc"
    else:
        filename += f"-nontrunc"
    filename += f"-n{n}-e_top_s{e_top_s}.pdf"

    plt.savefig("./continuous_plots/" + filename)
    plt.show()

In [ ]:
def M_and_M_prim_vs_beta():
    alpha = 0.6
    beta_values = np.linspace(0, 0.2, 500)
    gamma = 0.1
    delta = 0.000005
    n = 4039
    k = 0
    e_top_s = 1
    trunc=False

    cmp_funcs(alpha, beta_values, gamma, delta, k, n, e_top_s, trunc,
                     beta_values, r'$\beta$',
                     thm_sum_of_expressed_equilibrium_case2_M, r'$\mathbf{e}_M^\top \tilde \mathbf{z}^*$',
                     thm_sum_of_expressed_equilibrium_case2_M_prim, r'$\mathbf{e}_{Mprim}^\top \tilde \mathbf{z}^*$',
                     yscale='linear')

In [ ]:
def M_and_M_prim_vs_k():
    alpha = 0.51
    beta = 0.00010*0 #Why not summing to 1 when no impact from artificial sources?
    gamma = 0.00001
    delta = 0.000005
    n = 4039
    k = np.linspace(0, ((int)(np.minimum(alpha * n, (1-alpha)*n))), 500)
    e_top_s = 1
    trunc=False

    cmp_funcs(alpha, beta, gamma, delta, k, n, e_top_s, trunc,
                     k, r'$\k$',
                     thm_sum_of_expressed_equilibrium_case2_M, r'$\mathbf{e}_M^\top \tilde \mathbf{z}^*$',
                     thm_sum_of_expressed_equilibrium_case2_M_prim, r'$\mathbf{e}_{Mprim}^\top \tilde \mathbf{z}^*$',
                     thm_sum_of_expressed_equilibrium_case2_tot, r'$\mathbf{e}^\top \tilde \mathbf{z}^*$',
                     yscale='linear')

In [ ]:
M_and_M_prim_vs_k()